### Reranker from OCI Model Deployment
* https://github.com/run-llama/llama_index/blob/main/llama_index/postprocessor/cohere_rerank.py

In [1]:
import ads

from oci_utils import load_oci_config

from llama_index.schema import NodeWithScore, TextNode, QueryBundle

from oci_baai_reranker import OCIBAAIReranker
from config import RERANKER_ID

In [2]:
# this is the ID of the Model deployment
ID = RERANKER_ID

In [3]:
# load API keys from my config
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

In [4]:
reranker = OCIBAAIReranker(
    auth=api_keys_config, deployment_id=ID, region="eu-frankfurt-1"
)

2024-01-02 10:58:56,496 - INFO - Created OCI reranker client...
2024-01-02 10:58:56,497 - INFO - Region: eu-frankfurt-1...
2024-01-02 10:58:56,497 - INFO - Deployment id: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdyaokm6zawt3akgu3lr7u4hm4o4zrr64emfr3vi2qmzw2xa...


#### test the interface

In [5]:
query = "what is a panda?"

texts = [
    "World president",
    "The giant panda is a bear species endemic to China.",
    "It is a black and white animal endemic to China",
]

In [6]:
%%time

# evaluate texts against the query
response = reranker.rerank(query, texts)

CPU times: user 20.6 ms, sys: 4.47 ms, total: 25 ms
Wall time: 494 ms


In [7]:
response

[{'text': 'The giant panda is a bear species endemic to China.',
  'index': 1,
  'relevance_score': 4.247812747955322},
 {'text': 'It is a black and white animal endemic to China',
  'index': 2,
  'relevance_score': 1.590875506401062}]

* the first text is removed (top_n=2)
* strings are returned in decreasing relevance

#### Test compute_score

In [8]:
x = [[query, text] for text in texts]

In [9]:
reranker._compute_score(x)

{'prediction': [-9.481059074401855, 4.247812747955322, 1.590875506401062]}

#### Test the interface for llama_index

In [10]:
from oci_llama_reranker import OCILLamaReranker

In [11]:
llama_reranker = OCILLamaReranker(oci_reranker=reranker, top_n=2)

In [12]:
llama_reranker._rerank(query, texts, top_n=3)

[{'text': 'The giant panda is a bear species endemic to China.',
  'index': 1,
  'relevance_score': 4.247812747955322},
 {'text': 'It is a black and white animal endemic to China',
  'index': 2,
  'relevance_score': 1.590875506401062},
 {'text': 'World president',
  'index': 0,
  'relevance_score': -9.481059074401855}]

In [13]:
qb = QueryBundle(query_str=query)

list_nodes = [NodeWithScore(node=TextNode(text=text)) for text in texts]

In [14]:
llama_reranker.postprocess_nodes(nodes=list_nodes, query_bundle=qb)

[NodeWithScore(node=TextNode(id_='972233ab-2bc3-40d3-8270-442110709496', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='70045ac159f0f7146cbee52f9d7820c871954e19264fbc4cda958cbb1bba1efe', text='The giant panda is a bear species endemic to China.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=4.247812747955322),
 NodeWithScore(node=TextNode(id_='00ce9c76-f961-4bd1-b955-934a9f4e04d3', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='f32780c5b1e3206ac96c670b9512b36c8d7ccfe77c1e79cb62c17e14c41c324d', text='It is a black and white animal endemic to China', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1.590875506401062)]